In [1]:
import json
import ast
import pandas
from collections import defaultdict
import numpy as np

In [2]:
female_reviews = []
count_errors = 0
with open('review_buckets_female.txt') as data_file:    
    for line in data_file:
        try:
            female_reviews.append(ast.literal_eval(line))
        except:
            line = "{'"+line[2:]
            try:
                female_reviews.append(ast.literal_eval(line))
            except:
                count_errors+=1
print(str(len(female_reviews))+" female reviews read with "+str(count_errors)+ " error(s).")

# fixing key issues
for i in range(len(female_reviews)):
    review = female_reviews[i]
    for key in review.keys():
        if 'movie_id' in key:
            review['movie_id'] = review[key]
        if 'icon' in key:
            review['icon'] = review[key]
    female_reviews[i] = review
    
male_reviews = []
count_errors = 0
with open('review_buckets_male.txt') as data_file:    
    for line in data_file:
        try:
            male_reviews.append(ast.literal_eval(line))
        except:
            count_errors+=1
print(str(len(male_reviews))+" male reviews read with "+str(count_errors)+ " error(s).")

# fixing key issues
for i in range(len(male_reviews)):
    review = male_reviews[i]
    for key in review.keys():
        if 'movie_id' in key:
            review['movie_id'] = review[key]
        if 'icon' in key:
            review['icon'] = review[key]
    male_reviews[i] = review

64366 female reviews read with 0 error(s).
64366 male reviews read with 0 error(s).


In [14]:
movie_data = pandas.read_csv('movie_data_cleaned.csv')
movie_titles = [repr(x).split("\\", 1)[0].lower()[1:] for x in list(movie_data['movie_title'])]
movie_titles = [x[:-1] for x in movie_titles]
print("There are "+str(len(movie_titles))+" movies in imdb dataset")
print(np.shape(movie_data))

There are 3801 movies in imdb dataset
(3801, 45)


In [12]:
male_review_movie_list = [] # list of movie_ids for male reviews
male_movie_to_review = defaultdict(list) # movie_id -> lst of reviews
for i in male_reviews:
    if not str.isdigit(i['movie_id']):
        male_review_movie_list.append(i['movie_id'])
        male_movie_to_review[i["movie_id"]].append(i)
male_review_movie_list = set(male_review_movie_list)

female_review_movie_list = []
female_movie_to_review = defaultdict(list)
for i in female_reviews:
    if not str.isdigit(i['movie_id']):
        female_review_movie_list.append(i['movie_id'])
        female_movie_to_review[i["movie_id"]].append(i)
female_review_movie_list = set(female_review_movie_list)

In [15]:
review_movie_id_to_imdb_id = {} # converting id from rt to id from imdb

overlap_male_imdb = []
overlap_male_rt = []

for i in movie_titles:
    if i.replace(" ", "_") in male_review_movie_list:
        overlap_male_imdb.append(i)
        review_movie_id_to_imdb_id[i.replace(" ", "_")] = i
        overlap_male_rt.append(i.replace(" ", "_"))

overlap_female_imdb = []
overlap_female_rt = []

for i in movie_titles:
    if i.replace(" ", "_") in female_review_movie_list:
        overlap_female_imdb.append(i)
        overlap_female_rt.append(i.replace(" ", "_"))
        
overlap_intr = set(overlap_male_imdb) & set(overlap_female_imdb)
overlap_union = set(overlap_male_imdb) | set(overlap_female_imdb)

In [16]:
print(len(overlap_intr))
print(len(overlap_union))
print(len(set(overlap_female_rt)))

818
818
818


In [17]:
rt_movies = list(set(overlap_female_rt)) # all rt movies in overlap
movie_to_idx = {} # movie to index in rt_movies
for i in range(len(rt_movies)):
    movie_to_idx[rt_movies[i]] = i

In [18]:
reviewers = defaultdict(list)
for i in male_movie_to_review: # i is movie
    for j in male_movie_to_review[i]: # j is review within movie
        if (j["movie_id"] in rt_movies):
            j["gender"] = "m"
            reviewers[j["name"]].append(j)
for i in female_movie_to_review:
    for j in female_movie_to_review[i]:
        if (j["movie_id"] in rt_movies):
            j["gender"] = "f"
            reviewers[j["name"]].append(j)
reviewers_lst = [] # just names of each reviewer
for i in reviewers:
    reviewers_lst.append(i)

In [20]:
print("There are " + str(len(reviewers_lst)) + " reviewers total")

There are 2482 reviewers total


In [21]:
num_reviewers = len(reviewers)
reviewer_to_idx = {} # name -> idx in reviewers_lst
for i in range(len(reviewers_lst)):
    reviewer_to_idx[reviewers_lst[i]]=i
print(len(reviewers_lst))
print(num_reviewers)

2482
2482


In [22]:
review_matrix = np.zeros((num_reviewers,len(rt_movies)))
for i in reviewers:
    row = reviewer_to_idx[i]
    for j in reviewers[i]:
        col = movie_to_idx[j["movie_id"]]
        if (j["icon"] == "rotten"):
            review_matrix[row][col] = -1
        else:
            review_matrix[row][col] = 1

In [23]:
# review_matrix is reviewer by movie
count_reviews = np.zeros(num_reviewers)
for i in range(num_reviewers):
    for j in range(len(rt_movies)):
        if (review_matrix[i][j]==-1 or review_matrix[i][j]==1):
            count_reviews[i]+=1
print(count_reviews)

[ 14.   1.  52. ...,   1.   1.   2.]


In [24]:
review_row_idx_to_imdb_row = np.zeros(len(rt_movies))
count = 0
for i in range(len(rt_movies)):
    idx = movie_to_idx[rt_movies[i]]
    for j in range(len(movie_titles)):
        if movie_titles[j].replace(" ", "_") == rt_movies[i]:
            review_row_idx_to_imdb_row[i]=int(j)

In [25]:
review_row_idx_to_imdb_row

array([  2.25000000e+02,   1.36800000e+03,   2.60000000e+02,
         2.50300000e+03,   6.73000000e+02,   3.31300000e+03,
         1.38100000e+03,   7.98000000e+02,   2.43100000e+03,
         5.25000000e+02,   4.40000000e+01,   2.57700000e+03,
         6.99000000e+02,   8.59000000e+02,   1.37800000e+03,
         2.32200000e+03,   1.54800000e+03,   1.83000000e+02,
         6.17000000e+02,   2.55400000e+03,   2.10500000e+03,
         3.67400000e+03,   1.60000000e+02,   1.16800000e+03,
         7.46000000e+02,   1.90300000e+03,   3.98000000e+02,
         3.92000000e+02,   3.97000000e+02,   2.15600000e+03,
         1.46600000e+03,   3.18000000e+02,   2.00000000e+02,
         1.94000000e+02,   2.20700000e+03,   2.12000000e+03,
         6.61000000e+02,   2.80000000e+02,   2.73400000e+03,
         2.84300000e+03,   2.02900000e+03,   3.38600000e+03,
         1.47100000e+03,   2.11300000e+03,   2.45200000e+03,
         1.00000000e+01,   3.13000000e+02,   1.79600000e+03,
         3.65000000e+03,

In [26]:
np.shape(review_row_idx_to_imdb_row)

(818,)

In [27]:
np.shape(review_matrix)

(2482, 818)

In [28]:
np.shape(movie_data)

(3801, 45)